# Comparative Model Analysis

Cross-model comparison and integrated cardiac simulation insights.

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sys.path.insert(0, '..')

from quantro_simulator import (
    SimulationConfig, ModelType, OverlayMode,
    run_comprehensive_simulation, run_parameter_sweep
)

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Comprehensive Simulation

In [ ]:
print("Running comprehensive simulation...")
results_df = run_comprehensive_simulation()

print(f"\nTotal simulations: {len(results_df)}")
print(f"Models: {results_df['model'].unique()}")
print(f"Modes: {results_df['mode'].unique()}")
print(f"\nDataset preview:")
results_df.head(10)

## 2. Statistical Summary by Model

In [ ]:
summary_stats = results_df.groupby('model').agg({
    'val1': ['mean', 'std', 'min', 'max'],
    'val2': ['mean', 'std'],
    'val3': ['mean', 'std']
})

print("Summary Statistics by Model:")
print(summary_stats)

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for idx, val_col in enumerate(['val1', 'val2', 'val3']):
    ax = axes[idx]
    results_df.boxplot(column=val_col, by='model', ax=ax)
    ax.set_xlabel('Model', fontsize=11)
    ax.set_ylabel(val_col, fontsize=11)
    ax.set_title(f'Distribution of {val_col} by Model', fontsize=12)
    ax.get_figure().suptitle('')

plt.tight_layout()
plt.savefig('../artifacts/model_comparison_boxplot.png', dpi=300)
plt.show()

## 3. Overlay Mode Comparison

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

models = results_df['model'].unique()

for idx, model in enumerate(models):
    if idx >= 6:
        break
    
    model_data = results_df[results_df['model'] == model]
    
    for mode in model_data['mode'].unique():
        mode_data = model_data[model_data['mode'] == mode]
        axes[idx].plot(mode_data['lambda'], mode_data['val1'], 
                      'o-', label=mode, alpha=0.7, markersize=3)
    
    axes[idx].set_xlabel('Lambda', fontsize=10)
    axes[idx].set_ylabel('val1', fontsize=10)
    axes[idx].set_title(f'{model} Model', fontsize=11, fontweight='bold')
    axes[idx].legend(fontsize=8)
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../artifacts/overlay_mode_comparison.png', dpi=300)
plt.show()

## 4. Correlation Analysis

In [ ]:
# Correlation matrix
numeric_cols = ['lambda', 'val1', 'val2', 'val3']
corr_matrix = results_df[numeric_cols].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix: All Simulations', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig('../artifacts/correlation_matrix.png', dpi=300)
plt.show()

print("\nCorrelation coefficients:")
print(corr_matrix)

## 5. Model Performance Metrics

In [ ]:
# Calculate variability and sensitivity
model_metrics = []

for model in results_df['model'].unique():
    model_data = results_df[results_df['model'] == model]
    
    metrics = {
        'Model': model,
        'Mean val1': model_data['val1'].mean(),
        'Std val1': model_data['val1'].std(),
        'Range val1': model_data['val1'].max() - model_data['val1'].min(),
        'CV val1': model_data['val1'].std() / (model_data['val1'].mean() + 1e-10),
        'Lambda Sensitivity': np.corrcoef(model_data['lambda'], model_data['val1'])[0, 1]
    }
    model_metrics.append(metrics)

metrics_df = pd.DataFrame(model_metrics)
print("\nModel Performance Metrics:")
print(metrics_df.to_string(index=False))

# Radar chart
from math import pi

categories = ['Mean', 'Variability', 'Range', 'Sensitivity']
N = len(categories)

angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

for _, row in metrics_df.iterrows():
    values = [
        row['Mean val1'] / metrics_df['Mean val1'].max(),
        row['Std val1'] / metrics_df['Std val1'].max(),
        row['Range val1'] / metrics_df['Range val1'].max(),
        abs(row['Lambda Sensitivity'])
    ]
    values += values[:1]
    
    ax.plot(angles, values, 'o-', linewidth=2, label=row['Model'])
    ax.fill(angles, values, alpha=0.15)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, fontsize=11)
ax.set_ylim(0, 1)
ax.set_title('Normalized Model Metrics', fontsize=13, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
ax.grid(True)

plt.tight_layout()
plt.savefig('../artifacts/model_radar_chart.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Export Comprehensive Results

In [ ]:
# Save to CSV
output_path = '../comprehensive_results.csv'
results_df.to_csv(output_path, index=False)
print(f"✓ Results exported to {output_path}")

# Save metrics
metrics_path = '../model_metrics.csv'
metrics_df.to_csv(metrics_path, index=False)
print(f"✓ Metrics exported to {metrics_path}")

## Summary and Conclusions

### Key Findings:

1. **Model Diversity**: Each model captures different cardiac physiological aspects
2. **Parameter Sensitivity**: Varies significantly across models and overlay modes
3. **Overlay Effects**: Different modes provide distinct modulation strategies
4. **Integration**: Combined models offer comprehensive cardiac simulation

### Recommendations:

- Use FHN for electrophysiology studies
- Apply Poiseuille for hemodynamic modeling
- Leverage SIR for activation spread patterns
- Combine models for integrated cardiac function analysis

### Next Steps:

- Calibrate models with clinical data
- Implement patient-specific parameterization
- Develop real-time simulation capabilities
- Integrate with imaging data